# Integrated model for routing pollsters and vehicles

<div class="alert alert-block alert-info">
This notebook contains the code to build and solve the Integrated Vehicles and Polsters Routing Problem (IVPRP). 
</div>

In [1]:
from gurobipy import *

In [2]:
# Packages
import numpy  as np
import pandas as pd
import time
from collections import deque

In [3]:
# Aliases
append, arange, around, asarray, loadtxt, zeros = np.append, np.arange, np.around, np.asarray, np.loadtxt, np.zeros
DataFrame, concat = pd.DataFrame, pd.concat

In [4]:
# General functions for reading and writing
def Reader(nom):    return loadtxt('Instances/'+ nom + '.txt' )
def P_Reader(n):    return  Reader('Pollster_' + str(n))
def V_Reader(n):    return  Reader( 'Vehicle_' + str(n))
def S_Reader(n):    return  Reader( 'Service_' + str(n))
def T_Reader(n):    return  Reader(    'Time_' + str(n))

def Write(df,nom):  return DataFrame(df).to_csv('Instances/'+ nom +'.txt', header=None, index=None, sep=' ', mode='a')

## Basics

<div class="alert alert-block alert-warning">
First, basic data is registered:
    
* $n$ is the number of stores to be visited,
* $E$ is the set of available pollsters,
* $K$ is the set of available vehicles,
* $Q$ is the vehicle's capacity,
* $S$ is the number of days.
</div>

In [5]:
n = 50
E = arange(5);    K = arange(3);    Q = min(4,E.size);    S = arange(15)

<div class="alert alert-block alert-success">
Now we read the data associated with the instance.
</div>

In [6]:
Service  = S_Reader(n)
Poll     = P_Reader(n)
Vehicles = V_Reader(n)
Time     = T_Reader(n)

<div class="alert alert-block alert-warning">
The read matrices are processed:
    
* $d$ captures pollstering times,
* $t$ is the time that a pollster takes to walk among pairs of stores,
* $\tau$ is the time that vehicles take to move between pairs of stores,

* $[\rho_0,\rho_1]$ is the time window for breaks,
* $P$ is the length of the pause,
* $\beta$ is the time horizon limit.
</div>

In [7]:
d = append(around(Service, 2), 0.0)
t = around(Poll, 2)
τ = around(Vehicles, 2)
ρ_0, ρ_1, P, β = Time.ravel()

In [8]:
d[1:-1].sum()

1126.0

In [9]:
P

25.0

<div class="alert alert-block alert-warning">
At last costs are fixed:
    
* $\kappa_0$ is the daily cost of operations,
* $\kappa_1$ is the cost of hiring a vehicle,
* $\kappa_2$ is the cost of hiring a pollster.
</div>

In [10]:
κ_0 = 200.0
κ_1 = 100.0
κ_2 = 40.0

In [11]:
DataFrame(1*(t < τ[1:,1:])).replace({0:''}).head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,,,,,,,,,,,...,,,,,,,,,,
1,,,,,,,,,,,...,,,,,,,,,,
2,,,,,,,,,,,...,,,,,,,,,,
3,,,,,,,,,,,...,,,,,,,,,,
4,,,,,,,,,,,...,,,,,,,,,,


Finding an upper bound:

\begin{align}
    &\min_{(e\in E, k\in K, s\in S)} \kappa_0 (s+1) + \kappa_1 (k+1)(s+1) + \kappa_{2} (e+1)(s+1)
    \\
    \text{subject to} \qquad &
    \\
    & \dfrac{\left( \sum_{i\in C_-} t_{i,i+n} + (e+1) \times (s+1) \times P \right)}{(e+1)\times (s+1)} + \dfrac{ \left( \sum \max_{i} \tau_{} + \max_{j} \tau \right) }{ (k+1) \times (s+1) } \leq B_{\max}
    \\
    & (e+1) \leq Q (k+1)
\end{align}

In [12]:
Feas = {(e+1,k+1,s+1): (d.sum() + (e+1)*(s+1)*P)/((e+1)*(s+1)) 
            + (τ.max(axis=0) + τ.max(axis=1)).sum()/((k+1)*(s+1)*(1)) for e in E for k in K for s in S}
#{(e+1,k+1,s+1): (d.sum() + (e+1)*(s+1)*P)/((e+1)*(s+1)) + τ.sum()/((k+1)*(s+1)*(e+2)) for e in E for k in K for s in S}
#print({aa: (around(bb,2), bb < β) for aa,bb in Feas.items()})
print('')
Feas = {aa: κ_2 * aa[0] * aa[2] + κ_1 * aa[1] * aa[2] + κ_0 * aa[2] for aa,bb in Feas.items() if (bb < β) and (aa[1] <= aa[0] <= Q * (aa[1]))}
mE, mK, mS = min(Feas, key=Feas.get)
#print(Feas)
print('\nAt most there should be',mE,'pollsters,',mK,'vehicles, and',mS,'days.')



At most there should be 5 pollsters, 3 vehicles, and 2 days.


In [13]:
E = arange(mE);    K = arange(mK);    S = arange(mS)

## Graph

<div class="alert alert-block alert-warning">
The multigraph is built.
</div>

In [14]:
m = 2*n

# Domain for vehicles.                         Graph size:    (2*n) * (n + 1) + (2*n-1) * n = 4*(n**2) + n
dom_v  = [ (0,j)   for j in range(1,m+1) ] 
dom_v += [ (j,m+1) for j in range(1,m+1) ]
dom_v += [ (i,j)   for i in range(1,m+1) for j in range(1,m+1) if j not in [i,i-n]]

# Domain for pollsters.                        Graph size:    (n-1)**2 + (n-1) + n = n**2
dom_e  = [ (i,i+n) for i in range(  1,n+1) ]
dom_e += [ (i,j)   for i in range(1+n,m+1) for j in range(1,n+1) if j!=i-n ]

In [15]:
C_minus, C_plus, C_0, C_m, C = range(1,n+1), range(n+1,2*n+1),  range(0,n+1), range(1,2*n+2),  range(1,2*n+1)

## Integer Programming Model

<div class="alert alert-block alert-warning">
Now we are ready to build the IP model for IVPRP.
</div>

In [16]:
mo = Model()
x, y, z, b, f, w, B, u = {}, {}, {}, {}, {}, {}, {}, {}

Using license file /Users/andy/gurobi.lic
Academic license - for non-commercial use only


### Variables and Objective

In [17]:
# Pollster variables
## Nodes
b = mo.addVars( C_minus, E, S, vtype = 'B', name ='b')             # **begining**    n * |S|*|E|
f = mo.addVars( C_plus,  E, S, vtype = 'B', name ='f')             # **ending**      n * |S|*|E|
w = mo.addVars( C_minus, E, S, vtype = 'B', name ='w')             # **break**       n * |S|*|E|
## Arcs
x = mo.addVars( dom_e, E, S, vtype = 'B', name = 'x')              # **Walking paths**     n^2 * |S|*|E|

# Vehicle variables
y = mo.addVars( dom_v, K, S, vtype = 'B', name = 'y')               # **Vehicle-paths**     (4*n^2+n)*|K|*|S|
z = mo.addVars( dom_v, E, S, vtype = 'B', name = 'z')               # **t-paths**           (4*n^2+n)*|E|*|S|

# Time and days variables
B = mo.addVars( C, vtype = 'C', name = 'B', ub  = β )               # **In-Out timing**     2*n
u = mo.addVars( S, vtype = 'B', name = 'u', obj = κ_0)              # **Day**               |S|

mo.update()

Additional terms in objective

In [18]:
deque( (v.setAttr('obj', κ_1) for v in y.select(0,'*')), 0)
deque( (v.setAttr('obj', κ_2) for v in z.select(0,'*')), 0)
mo.setAttr('ModelSense', GRB.MINIMIZE)

In [19]:
mo.update()

### Constraints

<div class="alert alert-block alert-warning">
We begin with constraints for pollster routing, these are constraints $(1a)$ to $(1h)$.
</div>

In [20]:
# x & z vars interaction
start = time.time()
# 1a - Exclusive attention:      sum x[i,i+n] = 1
mo.addConstrs( (x.sum(i,i+n,'*') == 1 for i in C_minus), name='R-1a');

# 1b,c - Flow conservation:          sum_j x[j,i] - x[i,j] = b[i] - f[i]
mo.addConstrs( (x.sum('*',i,e,s) == x[i,i+n,e,s] - b[i,e,s] for i in C_minus for s in S for e in E), name='R-1b')
mo.addConstrs( (x.sum(i,'*',e,s) == x[i-n,i,e,s] - f[i,e,s] for i in C_plus for s in S for e in E), name='R-1c')

# 1d,e,f,g — Terminal pick-up and delivery:  sum_j z[i,j] <= 1
mo.addConstrs( (z.sum('*',i,e,s) <= 1.0 - x[i,i+n,e,s] + b[i,e,s] for i in C_minus for s in S for e in E), name='R-1d')
mo.addConstrs( (z.sum(i,'*',e,s) <= 1.0 - x[i-n,i,e,s] + f[i,e,s] for i in C_plus for s in S for e in E), name='R-1e')

mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == b[i,e,s] for i in C_minus for s in S for e in E), name='R-1f')
mo.addConstrs( (z.sum('*',i,e,s) - z.sum(i,'*',e,s) == -f[i,e,s] for i in C_plus for s in S for e in E), name='R-1g')

# 1h – One trip per day for each pollster
mo.addConstrs( (z.sum(0,'*',e,s) <= u[s] for s in S for e in E), name='R-1h')


end = time.time()
print('First block took {} seconds to compute.'.format(end-start))

mo.update()

First block took 0.1628739833831787 seconds to compute.


<div class="alert alert-block alert-warning">
Then we turn to vehicle routing, these are constraints $(2a)$ to $(2f)$.
</div>

In [21]:
# y & z vars interaction
start = time.time()
# 2a,b - Terminal arrivals:       sum_(j,k) y[i,j] = sum_e b[i] + f[i]
mo.addConstrs( (y.sum(i,'*','*',s) == b.sum(i,'*',s) for i in C_minus for s in S), name='R-2a')
mo.addConstrs( (y.sum(i,'*','*',s) == f.sum(i,'*',s) for i in C_plus for s in S), name='R-2b')
# 2c,d – Flow conservation:       sum_j y[i,j] - sum_j y[j,i] = 0
mo.addConstrs( (y.sum('*',i,k,s) == y.sum(i,'*',k,s) for i in C for k in K for s in S ), name='R-2c')
mo.addConstrs( (y.sum('*',2*n+1,k,s) == y.sum(0,'*',k,s) for k in K for s in S ), name='R-2d')
# 2e – One trip per day for each vehicle:     sum_i y[0,i] <= 1
mo.addConstrs( (y.sum(0,'*',k,s) <= u[s] for k in K for s in S ), name='R-2e')
# 2f – Capacity load:             sum_e z[i,j] <= Q sum_k y[i,j]
mo.addConstrs( (z.sum(i,j,'*',s) <= Q*y.sum(i,j,'*',s) for (i,j) in dom_v for s in S ), name='R-2f')

end = time.time()
print('Took {} seconds.'.format(end-start))

mo.update()

Took 0.5854108333587646 seconds.


<div class="alert alert-block alert-warning">
Next are time management and shift lengths $(3a)$ to $(3e)$.
</div>

In [22]:
M = β + max(τ.max(), t.max()) + 10. #1e+4

In [23]:
# B vars enforce connected paths
start = time.time()
# 3a,b – Arriving marker:  B[j] >= B[i] + t[i,j] + sum_s w[i] P - M(1 - sum_s x[i,j])
mo.addConstrs( (B[i+n] - B[i] - d[i] >= P*w.sum(i,'*') for i in C_minus ), name='R-3a')
mo.addConstrs( (B[j] - B[i+n] - t[i-1,j-1] >= -M*(1.0 - x.sum(i+n,j,'*')) 
               for i in C_minus for j in C_minus if j!=i ), name='R-3b')
## Trivial
mo.addConstrs( (B[i+n] - B[i] >= 0.0 for i in C_minus ), name='R-3-o')
# 3c — Arrival after transport: B[j] >= B[i] + τ_{i,j} - M(1 - sum_{s,k} y[i,j] )
mo.addConstrs( 
    (B[j] - B[i] + M*(1.0 - y.sum(i,j,'*')) >= τ[i % (n+1) + (1 if i > n else 0), j % (n+1) + (1 if j > n else 0)] 
     for (i,j) in dom_v if i!=0 and j!=2*n+1 ), name='R-3c')
# 3d — First transportation: B[i] >= tau_{0,i} - M(1 - sum_{s,k} y[0,i]^{k,s} )
mo.addConstrs( (B[i] >= τ[0, i % (n+1) + (1 if i > n else 0)] - β*(1.0 - y.sum(0,i,'*','*')) 
               for i in C ), name='R-3d')

# 3e — Arrival marks: B[2n+1]^{s} >= B[i] + tau_{i,2n+1} - M(1-y[i,2n+1])
mo.addConstrs( ( β * u[s] - B[i] >= τ[i % (n+1) + (1 if i > n else 0),0] - M*(1.0 - y.sum(i,2*n+1,'*',s))
               for i in C for s in S ), name='R-3e')
# 3f – There's no TW information available.
end = time.time()
print('Took {} seconds.'.format(end-start))

mo.update()

Took 0.1975250244140625 seconds.


<div class="alert alert-block alert-warning">
Lastly, pollster breaks are considered in $(4a)$ through $(4c)$.
</div>

In [24]:
# w vars interact
start = time.time()
# 4a — Breaks TW:   p0 sum_{e,s} w[i] <= B[i] + d[i] <= p1 + M( 1 - sum_s w[i] )
mo.addConstrs( (ρ_0*w.sum(i,'*') - B[i] - d[i] <= 0.0 for i in C_minus ), name='R-4a0')
mo.addConstrs( (B[i] + d[i] - ρ_1 - β*(1.0 - w.sum(i,'*')) <= 0.0 for i in C_minus ), name='R-4a1')
# 4b – One break per pollster:     w[i] <=  x[i,j]
mo.addConstrs( (w[i,e,s] <= x[i,i+n,e,s] for i in C_minus for e in E for s in S ), name='R-4b')
# 5c — Mandatory breaks:   sum_i w[i] = sum_j z[0,j]
mo.addConstrs( (w.sum('*',e,s) == z.sum(0,'*',e,s) for e in E for s in S ), name='R-4b')

end = time.time()
print('Took {} seconds.'.format(end-start))

mo.update()

Took 0.019819021224975586 seconds.


### Symmetry breaking inequalities

<div class="alert alert-block alert-warning">
Constraints $(5a)$ to $(5l)$ are added here.
</div>

In [25]:
# More
start = time.time()

mo.addConstr( z.sum(0,'*',0,0) == 1 , name='R-5a')
mo.addConstr( y.sum(0,'*',0,0) == 1 , name='R-5d')

if E.size > 1:
    mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e-1,s) for e in E for s in S if e > 0 ), name='R-5b')
    
    if S.size > 1:
        mo.addConstrs( 
            (b.sum('*',e,s) <= n - quicksum(b[i,e-1,r] for i in C_minus for r in S if r<=s-1) 
             for e in E for s in S if e > 0 if s > 0 ), name='R-5k')
        mo.addConstrs( 
            (f.sum('*',e,s) <= n - quicksum(f[i,e-1,r] for i in C_plus for r in S if r<=s-1) 
             for e in E for s in S if e > 0 if s > 0 ), name='R-5l')
        
if S.size > 1:
    mo.addConstrs( (z.sum(0,'*',e,s) <= z.sum(0,'*',e,s-1) for e in E for s in S if s > 0 ), name='R-5c')
    mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k,s-1) for k in K for s in S if s > 0 ), name='R-5f')
    mo.addConstrs( (b.sum(i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5g')
    mo.addConstrs( (f.sum(i,'*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5h')
    mo.addConstrs( (x.sum('*',i,'*',s) <= 1.0 - b.sum(i,'*',s-1) for i in C_minus for s in S if s > 0 ), name='R-5i')
    mo.addConstrs( (x.sum(i,'*','*',s) <= 1.0 - f.sum(i,'*',s-1) for i in C_plus  for s in S if s > 0 ), name='R-5j')
    mo.addConstrs( (u[s] <= u[s-1] for s in S if s > 0), name='R-5m')
    
if K.size > 1:
    mo.addConstrs( (y.sum(0,'*',k,s) <= y.sum(0,'*',k-1,s) for k in K for s in S if k > 0 ), name='R-5e')
    
end = time.time()
print('SI: Took {} seconds.'.format(end-start))

SI: Took 0.04883599281311035 seconds.


---

In [107]:
Λ   = [ np.append(a, arange(a.size + b + 1, (b+1)*a.size + 1) ) for b,a in enumerate([(E+1) + s for s in S]) ]
# [ arange(s+1, (s+1)*E.size) for s in S ]
Λ_β = {λ: d.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d.sum()/λ <= β - P}
print(Λ_β)

{6: 187.66666666666666, 7: 160.85714285714286, 8: 140.75, 9: 125.11111111111111, 10: 112.6}


In [108]:
{λ: d.sum()/λ for λ in np.unique(np.concatenate(Λ)) if d.sum()/λ <= β - (P + τ[0,:][1:].min() + τ[:,0][1:].min())}

{6: 187.66666666666666,
 7: 160.85714285714286,
 8: 140.75,
 9: 125.11111111111111,
 10: 112.6}

In [109]:
F_2, T_1 = max(Λ_β.items(), key=lambda x: x[1]) 
F_1 = next(a for a,b in enumerate(Λ) if F_2 in b) + 1
F_3 = [k for k in arange(F_1, K.size * F_1 + 1) if F_2 <= Q * k].pop(0)

In [110]:
print('Lower bounds found: |S| >=', F_1, '|K| >=',F_3, '|E| >=', F_2)

Lower bounds found: |S| >= 2 |K| >= 2 |E| >= 6


In [30]:
# Additional bound over the number of days
mo.addConstr( u.sum() >= F_1)
# Additional bound over the number of required pollsters
mo.addConstr(z.sum(0,'*','*','*') >= F_2);
# Additional bound over the number of required vehicles
mo.addConstr(y.sum(0,'*','*','*') >= F_3);

In [31]:
mo.addConstrs( (z.sum(0,'*',0,s) == u[s] for s in S) )
mo.addConstrs( (y.sum(0,'*',0,s) == u[s] for s in S) );

In [32]:
#mo.addConstrs( (z.sum(0,'*','*',s) <= z.sum(0,'*','*',s-1) for s in S if s>0) )
mo.addConstrs( (y.sum(0,'*','*',s) <= y.sum(0,'*','*',s-1) for s in S if s>0) );

In [33]:
deque( (v.setAttr('BranchPriority', 10) for v in y.values()), 0);
#deque( (v.setAttr('BranchPriority', 10) for v in z.values()), 0)

## Optimization

In [34]:
# Piece of solution

# Day 1
# First half
b[4,1,0].lb = 1
b[5,0,0].lb = 1
b[9,0,0].lb = 1
f[15,0,0].lb = 1
f[16,1,0].lb = 1
f[19,0,0].lb = 1
x[4,14,1,0].lb = 1
x[5,15,0,0].lb = 1
x[6,16,1,0].lb = 1
x[8,18,1,0].lb = 1
x[9,19,0,0].lb = 1
x[14,8,1,0].lb = 1
x[18,6,1,0].lb = 1

y[0,4,1,0].lb = 1
y[0,5,0,0].lb = 1
#y[16,21,1,0].lb = 1
#y[19,21,0,0].lb = 1
#y[4,16,1,0].lb = 1
y[5,15,0,0].lb = 1
#y[9,19,0,0].lb = 1
y[15,9,0,0].lb = 1

z[0,4,1,0].lb = 1
z[0,5,0,0].lb = 1
#z[16,21,1,0].lb = 1
#z[19,21,0,0].lb = 1
z[15,9,0,0].lb = 1

# Second half
b[4,1,0].lb = 1
b[5,0,0].lb = 1
b[9,0,0].lb = 1
f[15,0,0].lb = 1
f[16,1,0].lb = 1
f[19,0,0].lb = 1
x[4,14,1,0].lb = 1
x[5,15,0,0].lb = 1
x[6,16,1,0].lb = 1
x[8,18,1,0].lb = 1
x[9,19,0,0].lb = 1
x[14,8,1,0].lb = 1
x[18,6,1,0].lb = 1

#y[0,4,1,0].lb = 1
#y[0,5,0,0].lb = 1
#y[16,21,1,0].lb = 1
#y[19,21,0,0].lb = 1
#y[4,16,1,0].lb = 1
#y[5,15,0,0].lb = 1
y[9,19,0,0].lb = 1
y[15,9,0,0].lb = 1
#z[0,4,1,0].lb = 1
#z[0,5,0,0].lb = 1
#z[16,21,1,0].lb = 1
#z[19,21,0,0].lb = 1
z[15,9,0,0].lb = 1


# Day 2
# First half



# Second half




Pick the best solution (if any) from below.

In [35]:
mo.reset()
mo.resetParams()
mo.Params.VarBranch = 1
mo.Params.BranchDir = 0
mo.Params.TimeLimit = 360
#mo.Params.Cuts = 0
mo.optimize()

Discarded solution information
Reset all parameters
Changed value of parameter VarBranch to 1
   Prev: -1  Min: -1  Max: 3  Default: -1
Parameter BranchDir unchanged
   Value: 0  Min: -1  Max: 1  Default: 0
Changed value of parameter TimeLimit to 360.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 3327 rows, 7366 columns and 35800 nonzeros
Model fingerprint: 0x86579e2e
Variable types: 28 continuous, 7338 integer (7338 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+01, 2e+02]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 2e+02]
Presolve removed 3327 rows and 7366 columns
Presolve time: 0.03s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.07 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 960 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.600000000000e+02, best bound 9.6000000

In [53]:
# Parametrised
mo.reset()
mo.resetParams()
#mo.setParam('Symmetry', 2) #mo.setParam('BranchDir', 1); mo.setParam('Heuristics',1)

mo.Params.MIPFocus = 1;    
mo.Params.Heuristics = 0.33
mo.Params.Cuts = 3;      #<- w/o finds some feasible solutions 
mo.Params.Method = 2

mo.Params.SimplexPricing = 3
mo.Params.CutAggPasses = 12;    mo.Params.CutPasses = 12;   mo.Params.PrePasses = 8

mo.Params.ImproveStartTime = 100


mo.setParam('Presolve', 2)
mo.Params.GURO_PAR_PREPROBE = 3
mo.Params.PreSparsify = 1
mo.Params.PrePasses = 500 # best solution so far w/ 500

mo.Params.TimeLimit = 10*60
mo.optimize()

Discarded solution information
Reset all parameters
Changed value of parameter MIPFocus to 1
   Prev: 0  Min: 0  Max: 3  Default: 0
Changed value of parameter Heuristics to 0.33
   Prev: 0.05  Min: 0.0  Max: 1.0  Default: 0.05
Changed value of parameter Cuts to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter Method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Changed value of parameter SimplexPricing to 3
   Prev: -1  Min: -1  Max: 3  Default: -1
Changed value of parameter CutAggPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter CutPasses to 12
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter PrePasses to 8
   Prev: -1  Min: -1  Max: 2000000000  Default: -1
Changed value of parameter ImproveStartTime to 100.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Presolve to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter GURO_PAR_PREPROBE to 3
   Prev

  1494   264  480.00000   48   22  820.00000  480.00000  41.5%  44.8  225s
  1522   277  480.00000   53   57  820.00000  480.00000  41.5%  44.3  231s
  1584   275 infeasible   56       820.00000  480.00000  41.5%  43.3  236s
  1635   281  480.00000   43   33  820.00000  480.00000  41.5%  42.6  241s
  1677   287  480.00000   48   16  820.00000  480.00000  41.5%  42.0  247s
  1710   293  480.00000   51   68  820.00000  480.00000  41.5%  41.7  250s
  1749   298 infeasible   48       820.00000  480.00000  41.5%  41.2  256s
  1803   306 infeasible   50       820.00000  480.00000  41.5%  40.7  262s
  1827   308 infeasible   54       820.00000  480.00000  41.5%  40.4  266s
  1882   305 infeasible   53       820.00000  480.00000  41.5%  39.9  273s
  1912   307 infeasible   54       820.00000  480.00000  41.5%  39.6  278s
  1942   307  480.00000   38   36  820.00000  480.00000  41.5%  39.2  281s
  1992   305  480.00000   46   58  820.00000  480.00000  41.5%  38.8  286s
  2014   310 infeasible  

If fixing some variables generates a feasible solution, we can test against it:

In [50]:
deque( (v.setAttr('Start', v.x) for v in mo.getVars()), 0);
deque( (v.setAttr('LB', 0.0) for v in mo.getVars()), 0);

deque([])

## Results

<div class="alert alert-block alert-warning">
We identify variables that have assigned a non-zero value.
</div>

In [47]:
X = tupledict({nn: v for nn, v in x.items() if v.x > 0.0})
Y = tupledict({nn: v for nn, v in y.items() if v.x > 0.0})
Z = tupledict({nn: v for nn, v in z.items() if v.x > 0.0})

A = tupledict({nn: v for nn, v in b.items() if v.x > 0.0})
F = tupledict({nn: v for nn, v in f.items() if v.x > 0.0})
W = tupledict({nn: v for nn, v in w.items() if v.x > 0.0})
U = tupledict({nn: v for nn, v in u.items() if v.x > 0.0})

The last dictionary, `U`, contains which days are active. As a result, we can store only those days. Moreover, this also simplifies iterations. A similar treatment can be done with `E` and `K`.

In [48]:
Active_Days      = U.keys()
Active_Pollsters = {v[1] for v in A.keys()}
Active_Vehicles  = {v[2] for v in Y.keys()}

In [49]:
print('A solution was found with',len(Active_Pollsters),'pollsters,',len(Active_Vehicles),'vehicles, and',len(Active_Days),'days.')

A solution was found with 2 pollsters, 2 vehicles, and 1 days.


In [50]:
Out_File = 'Instances/Results_Fixing_P-2-a-b-'+ str(n) +'.xlsx'

In [51]:
with pd.ExcelWriter(Out_File) as writer:
    # General results
    Hoja = DataFrame({'0':['Active days','Active pollsters','Active vehicles', 'Objective','GAP','Time'], 
                      '1':[len(U), len(Active_Pollsters), len(Active_Vehicles), 
                               mo.ObjVal, str(around(mo.MIPGap * 100,2)) + ' %', around(mo.Runtime,2)]})
    Hoja.to_excel(writer, 'Summary', header=False, index= False)
    writer.sheets['Summary'].set_column('A:A', 15)
    
    # Pollster routing
    X_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
    for days in Active_Days:
        X_day = (v[:-1] for v in X.keys() if v[-1] == days)
        for pairs in X_day:
            coords = tuple([vv if vv <= n else vv%n if vv<2*n else n for vv in pairs[:-1] ])
            X_visits[days][coords] = pairs[-1] + 1
            
    Hoja = concat([DataFrame(X_visits[days]) for days in Active_Days], axis=1).replace({0:''})
    Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
    Hoja.to_excel(writer, 'Pollster routing')
    
    # Vehicle routing
    Y_visits = [zeros([n+1,n+1], dtype=int) for days in Active_Days]
    for days in Active_Days:
        Y_day = (v[:-1] for v in Y.keys() if v[-1] == days)
        for pairs in Y_day:
            coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
            Y_visits[days][coords] = pairs[-1] + 1
    
    Hoja = concat([DataFrame(Y_visits[days]) for days in Active_Days], axis=1).replace({0:''})
    Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
    Hoja.to_excel(writer, 'Vehicle routing')
    
    # Shared routing
    Z_visits = [zeros([n+1,n+1], dtype='<U'+str(2*n)) for days in Active_Days]
    for days in Active_Days:
        Z_day = (v[:-1] for v in Z.keys() if v[-1] == days)
        for pairs in Z_day:
            coords = tuple([vv if vv <= n else vv%n if vv<2*n else n if vv == 2*n else 0 for vv in pairs[:-1] ])
            if Z_visits[days][coords] == '':
                Z_visits[days][coords] = str(pairs[-1] + 1)
            else:
                Z_visits[days][coords] += ', ' + str(pairs[-1] + 1)
    
    Hoja = concat([DataFrame(Z_visits[days]) for days in Active_Days], axis=1).replace({0:''})
    Hoja.columns = [str(v) + ' day ' + str(days) for days in Active_Days for v in C_0 ]
    Hoja.to_excel(writer, 'Shared routing')
    
    # Times and breaks
    W_breaks = [ zeros(n, dtype=int) for days in Active_Days]
    for days in Active_Days:
        for v in (v[:-1] for v in W.keys() if v[-1] == days):
            W_breaks[days][v[0]-1] = v[1]+1
    
    Hoja = DataFrame({'Time i':[ B[i].x for i in C_minus ], 'Time i+n':[ B[i].x for i in C_plus ]})
    Hoja = concat([Hoja]+[DataFrame(W_breaks[days]) for days in Active_Days], axis=1).replace({0:''})
    Hoja.columns = list(Hoja.columns[:2]) + ['break day '+str(days) for days in Active_Days]
    Hoja.index = arange(1,n+1)
    Hoja.to_excel(writer, 'Times and breaks per pollster')
    writer.sheets['Times and breaks per pollster'].set_column('D:AA', 15)

---